In [250]:
import pandas as pd
import numpy as np
import pycountry
import countrydict
import province_to_regions
import re
from collections import defaultdict

In [251]:
df = pd.read_excel("emdat_public_2023_02_14_query_uid-AJtVn9.xlsx",skiprows = 6)
df = df[df['Insured Damages (\'000 US$)'].isna() == False]


In [252]:
#A semi-automated way to cluster some rows when the country spans multiple regions
def get_region(iso,location):
    #If the country only spans one region, we can get it directly, otherwise we need to look at the provinces/states
    regions = countrydict.countries_to_regions[iso]
    if(isinstance(regions,str)):
        return regions
    else:
        dict = province_to_regions.provinces_to_region[iso]
        for province,region in dict.items():
            if(location.find(province) != -1):
                return region
            #The china dictionary is structured differently since the majority of provinces are in the EAS region. We only need to consider tibet and xinjiang seperately
            if(iso == "CHN"):
                return "EAS"
        return "None"
    
df['Location'].fillna("",inplace=True)
df['Region'] = df.apply(lambda row: get_region(row.ISO,row.Location),axis = 1)

In [253]:
#Cluster 0 corresponds to losses that need to be considered or manually
df['Cluster'] = df['Region'].map(lambda x: countrydict.regions_to_cluster[x] if (isinstance(x,str)) else 0)

In [254]:
#df[df.Cluster == 0].to_excel("manual_clusters.xlsx")
#About 50 events need to be considered manually now

In [255]:
manual_clusters = pd.read_excel("manual_clusters.xlsx")

In [256]:
manual_clusters = manual_clusters.set_index(manual_clusters['Unnamed: 0'])
manual_clusters.drop(['Unnamed: 0'],axis = 1);

In [257]:
df.update(manual_clusters)
df['Cluster'] = df['Region'].map(lambda x: countrydict.regions_to_cluster[x])

In [258]:
#Incorporating GDP data


WEO_GDP = pd.read_csv("WEO_Data.csv",encoding='ISO-8859-1',skiprows = [197,198])
WB_GDP = pd.read_csv("World_Bank_Data.csv",skiprows = [267,268,269,270,271])

WEO_GDP = WEO_GDP.set_index(WEO_GDP['ISO']);
WB_GDP = WB_GDP.set_index(WB_GDP['Country Code']);
WEO_GDP = WEO_GDP.to_dict('index')
WB_GDP = WB_GDP.to_dict('index')

In [259]:
for key in WEO_GDP:
    WEO_GDP[key] = defaultdict(int,WEO_GDP[key])
for key in WB_GDP:
    WB_GDP[key] = defaultdict(int,WB_GDP[key])
WEO_GDP = defaultdict(lambda: defaultdict(int), WEO_GDP)
WB_GDP = defaultdict(lambda: defaultdict(int),WB_GDP)

In [260]:
def get_WEO_GDP(ISO,year):
    return WEO_GDP[ISO][year]
def get_WB_GDP(ISO,year):
    y = year + " [YR" + year + "]"
    return WB_GDP[ISO][y]

In [261]:
df['Year'] = (df['Year']).astype(int).astype(str)

In [262]:
df['WEO GDP'] = df.apply(lambda row: get_WEO_GDP(row.ISO,row.Year),axis = 1)
df['WB GDP'] = df.apply(lambda row: get_WB_GDP(row.ISO,row.Year),axis = 1)

In [263]:
df['WEO GDP'] = ((df['WEO GDP'].str.replace(',','')).astype(float))*1e9
df['WEO GDP'].fillna(0,inplace = True)

In [264]:
df['WB GDP'] = ((df['WB GDP'].replace('..','0'))).astype(float)
df['WB GDP'].fillna(0,inplace = True)

In [265]:
def modified_average(x,y):
    return ((x > 0) and (y > 0))*(x + y)/2 + ((x == 0) and (y > 0))*(y) + ((x > 0) and (y == 0))*x

In [266]:
df['GDP'] = df.apply(lambda row: modified_average(row['WEO GDP'],row['WB GDP']),axis = 1)

In [267]:
#17 GDPs are missing - need to be considered separately. Mostly in caribbean territories or countries that no longer exist.
df[df['GDP'] == 0].to_excel("manual_GDP_entry.xlsx")